### Who are the poor?

Attempt to replicate excellent chart by Matt Bruenig

----

Source: CPS ASEC

Variables:

- Person record:       PRECORD
- Age:                 A_AGE
- Person weight:       MARSUPWT 
- Poverty status:      PERLIS
- Why not full-year:   PYRSN
- Why not employed:    RSNNOTW
- Disability:          PRDISFLG
- Weeks unemployed:    NWLKWK
- Full-year worker:    WEWKRS

[Data dictionary](https://thedataweb.rm.census.gov/pub/cps/march/06_ASEC_2019-Data_Dictionary_Full.pdf)

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
variables = ['A_AGE', 'MARSUPWT', 'A_LFSR', 'PERLIS', 'PYRSN', 
             'NWLKWK', 'WEWKRS', 'RSNNOTW', 'SPM_Poor',
             'PRDISFLG', 'PRECORD', 'SPM_Weight']

dictfile = 'data/persfmt.txt'
dd = open(dictfile).read()
p = f'({"|".join(variables)})\s+(\d+)\s+(\d+)\s'
cols = {name: (int(start) - 1, int(start) - 1 + int(length)) 
        for name, length, start in re.findall(p, dd)}

datafile = 'data/asec2019_pubuse.dat'
df = (pd.read_fwf(datafile, 
                  colspecs=list(cols.values()), 
                  header=None, 
                  names=cols.keys())
        .query('PRECORD == 3 and PERLIS == 1 and MARSUPWT > 0'))

In [3]:
df['CAT'] = (np.where(df.A_AGE < 18, 'Children', 
             np.where(df.A_AGE > 64, 'Elderly', 
             np.where(((df.PRDISFLG == 1) | (df.PYRSN == 1) | (df.RSNNOTW == 1)), 'Disabled', 
             np.where(((df.PYRSN == 3) | (df.RSNNOTW == 4)), 'Student', 
             np.where(((df.PYRSN == 2) | (df.RSNNOTW == 3)), 'Carers',
             np.where(((df.PYRSN == 4) | (df.RSNNOTW == 2)), 'Nonelderly Retired',  
             np.where(((df.PYRSN == 5) | (df.RSNNOTW == 5) | (df.NWLKWK.astype(int) > 0)), 'Unemployed',
             np.where(df.WEWKRS.isin([1, 2]), 'Fully Employed', 'Other')))))))))

In [4]:
df.groupby('CAT').MARSUPWT.sum() / df.MARSUPWT.sum() * 100

CAT
Carers                 8.254535
Children              32.022004
Disabled              15.609597
Elderly               13.313400
Fully Employed         9.827651
Nonelderly Retired     3.188996
Other                  9.701647
Student                6.206497
Unemployed             1.875674
Name: MARSUPWT, dtype: float64